<a href="https://colab.research.google.com/github/ravi-gopalan/DAND_Data_Wrangling/blob/master/creating_entities.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [6]:
from google.colab import drive
drive.mount('/gdrive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /gdrive


In [7]:
!pip install sparqlwrapper
!pip install wikipedia

     |████████████████████████████████| 348kB 4.0MB/s 
     |████████████████████████████████| 51kB 5.7MB/s 
  Created wheel for wikipedia: filename=wikipedia-1.4.0-cp36-none-any.whl size=11686 sha256=6c88e02faa80e33a0c7d28303c8adc4c63af17d1170c45d0e2880d302c3eac92
  Stored in directory: /root/.cache/pip/wheels/87/2a/18/4e471fd96d12114d16fe4a446d00c3b38fb9efcb744bd31f4a
Successfully built wikipedia


In [0]:
import pandas as pd
import numpy as np
import json
from SPARQLWrapper import SPARQLWrapper, JSON
import re
from collections import Counter
import time
import itertools
import wikipedia
import requests
from pandas.io.json import json_normalize

from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

In [43]:
cd '/gdrive/My Drive/abv_reviews'

/gdrive/My Drive/abv_reviews


In [0]:
def get_results(endpoint_url, query):
    sparql = SPARQLWrapper(endpoint_url)
    sparql.setQuery(query)
    sparql.setReturnFormat(JSON)
    result = sparql.query()

    processed_results = json.load(result.response)
    cols = processed_results['head']['vars']

    out = []
    for row in processed_results['results']['bindings']:
      item = []
      for c in cols:
        item.append(row.get(c, {}).get('value'))
      out.append(item)
    return pd.DataFrame(out, columns = cols)

In [0]:
def query_and_process_results(url, query_parameter):

  q1 = 'SELECT ?item ?itemLabel WHERE {?item wdt:P279 wd:'
  q2 = '. SERVICE wikibase:label { bd:serviceParam wikibase:language "[AUTO_LANGUAGE],en". }}'

  query = q1 + query_parameter + q2
#  print(query)

  df = get_results(url, query)
  df.shape
  df.head()

  res_list =  sorted([re.sub("q[0-9]+","",item.lower()) for item in df.groupby(['itemLabel'])['item'].agg('count').index.values.tolist()])
  res_list = sorted([re.sub("\-", " ",item) for item in res_list])
  res_list = [item for item in res_list if not re.findall("[0-9]+",item)]
  res_counter = Counter(res_list)
  res_list = []
  for key in res_counter.keys():
    if len(key) != 0:
      res_list.append(key)

  print(len(res_list), res_list)

  return df, res_list, res_counter

In [0]:
def get_wikidata_id(col):
  rx = re.compile(r'(http\:\/\/www\.wikidata\.org\/entity\/)(Q[0-9]+)')
  m = rx.match(col)
  if m is not None:
    return m.group(2)



def get_wikidata_description(row):
  base_string = 'https://www.wikidata.org/w/api.php?action=wbgetentities&sites=enwiki&languages=en&format=json&props=descriptions&ids='
  query_string = base_string + row
#  print(query_string)
  r = requests.get(query_string)
  return json.loads(r.content.decode('utf-8'))['entities'][row]

def get_wikidata_label(row):
  base_string = 'https://www.wikidata.org/w/api.php?action=wbgetentities&sites=enwiki&languages=en&format=json&props=labels&ids='
  query_string = base_string + row
#  print(query_string)
  r = requests.get(query_string)
  return json.loads(r.content.decode('utf-8'))['entities'][row]

In [0]:
def dump_jsonl(data, output_path, append=False):
    """
    Write list of objects to a JSON lines file.
    """

    mode = 'a+' if append else 'w'
    with open(output_path, mode, encoding='utf-8') as f:
        for line in data:
          json_record = json.dumps(line, ensure_ascii=False)
          f.write(json_record + '\n')
#    print('Wrote {} records to {}'.format(len(data), output_path))


def load_jsonl(input_path) -> list:
    """
    Read list of objects from a JSON lines file.
    """
    data = []
    with open(input_path, 'r', encoding='utf-8') as f:
        for line in f:
            data.append(json.loads(line.rstrip('\n|\r')))
    print('Loaded {} records from {}'.format(len(data), input_path))
    return data

In [0]:
endpoint_url = "https://query.wikidata.org/sparql"

In [0]:
cat_list = ['noodle', 'soup', 'rice', 'salad','dish', 'drink','hot_bev', 'coffee']
query_parameter_list = ['Q192874','Q41415', 'Q21976260', 'Q9266', 'Q746549', 'Q40050', 'Q19359564', 'Q37756327']
df_list = []
result_lists = []
result_counters = []


for cat, query_parameter in zip(cat_list, query_parameter_list):
  print("Category query in progress for {}".format(cat))
  df, r_list, r_counter = query_and_process_results(endpoint_url, query_parameter)
  df_list.append(df)
  result_lists.append(r_list)
  result_counters.append(r_counter)
  print("Category {} - completed".format(cat))
  time.sleep(2)

In [0]:
for df in df_list:
  df['wikidata_entity_id'] = df['item'].apply(get_wikidata_id)
  df['desc_detail'] = df['wikidata_entity_id'].apply(get_wikidata_description)
  df['label_detail'] = df['wikidata_entity_id'].apply(get_wikidata_label)


In [0]:
for cat, df in zip(cat_list, df_list):
  temp = 'df_'+cat+'.csv'
  df = pd.concat([df,json_normalize(df['desc_detail'],max_level=2),json_normalize(df['label_detail'],max_level=2)],axis=1)
  df.to_csv(temp)

In [0]:
def get_wikipedia_page_summaries(input_list, output_jsonl_file,lang='en'):
  wikipedia.set_lang(lang)
  output_list = []
  output_issues_list = []

  for item in input_list:
    time.sleep(0.5)
    wiki_dict = {}
    try:
      wiki_dict['name'] = item
      wiki_dict['page'] = wikipedia.page(item).url
      wiki_dict['summary'] = wikipedia.summary(item)
      wiki_dict['image_ref'] = wikipedia.page(item).images[0]
    except:
      output_issues_list.append(item)
    output_list.append(wiki_dict)
  for item in output_list:
    dump_jsonl([item],output_jsonl_file, append=True)
  return output_issues_list

In [0]:
def remove_duplicate_list_items(input_list):
  input_list = sorted([re.sub("\-", " ",item.lower()) for item in input_list])
  c = Counter(input_list)
  output_list = []
  for key in c.keys():
    if len(key) != 0:
      output_list.append(key)
  
  return output_list

In [21]:
herbs_list = remove_duplicate_list_items(herbs_list)
print(len(herbs_list),herbs_list)

51 ['angelica', 'basil', 'bay leaf', 'boldo', 'bolivian coriander', 'borage', 'chervil', 'chives', 'cicely', 'cilantro', 'coriander', 'cress', 'culantro', 'curry leaf', 'dill', 'epazote', 'garlic chives', 'hemp', 'hoja santa', 'holy basil', 'houttuynia cordata', 'hyssop', 'indian bay leaf', 'jimbu', 'kinh gioi', 'lavender', 'lemon balm', 'lemon grass', 'lemon myrtle', 'lemon verbena', 'limnophila aromatica', 'lovage', 'marjoram', 'mint', 'mitsuba', 'mugwort', 'oregano', 'parsley', 'perilla', 'rosemary', 'rue', 'sage', 'sansho', 'savory', 'shiso', 'sorrel', 'tarragon', 'thai basil', 'thyme', 'vietnamese coriander', 'woodruff']


In [22]:
unresolved_herbs_list = get_wikipedia_page_summaries(herbs_list, 'herbs_list.jsonl',lang='en')
unresolved_herbs_list

/usr/local/lib/python3.6/dist-packages/wikipedia/wikipedia.py:389: UserWarning: No parser was explicitly specified, so I'm using the best available HTML parser for this system ("lxml"). This usually isn't a problem, but if you run this code on another system, or in a different virtual environment, it may use a different parser and behave differently.

The code that caused this warning is on line 389 of the file /usr/local/lib/python3.6/dist-packages/wikipedia/wikipedia.py. To get rid of this warning, pass the additional argument 'features="lxml"' to the BeautifulSoup constructor.

  lis = BeautifulSoup(html).find_all('li')


['culantro',
 'jimbu',
 'kinh gioi',
 'mint',
 'sage',
 'sansho',
 'savory',
 'woodruff']

In [0]:
def fix_dataframe(input_jsonl_file):
  df = pd.read_json(input_jsonl_file, lines=True)
  df.shape
  df.head()

  df[df.duplicated(['name'],False)]

  output_list = remove_duplicate_list_items(list(df[df['page'].isna()]['name']))
  print(len(output_list),output_list)

  output_list = [re.sub("q[0-9]+","",item.lower()) for item in output_list]
  output_list = sorted([re.sub("\-", " ",item) for item in output_list])

  df.dropna(axis=0,how='all',subset=['page','summary','image_ref'],inplace=True)
  df.drop_duplicates(inplace=True)
  df.shape
  df.head()

  return df

In [24]:
df_check_herbs = fix_dataframe('herbs_list.jsonl')
df_check_herbs.to_csv('df_check_herbs.csv')

7 ['culantro', 'kinh gioi', 'mint', 'sage', 'sansho', 'savory', 'woodruff']


In [31]:
df_check_herbs.head(50)

,name,page,summary,image_ref
0,angelica,https://en.wikipedia.org/wiki/Angelica,Angelica is a genus of about 60 species of tal...,https://upload.wikimedia.org/wikipedia/commons...
1,basil,https://en.wikipedia.org/wiki/Basil,"Basil (, also US: ; Ocimum basilicum), also ca...",https://upload.wikimedia.org/wikipedia/commons...
2,bay leaf,https://en.wikipedia.org/wiki/Bay_leaf,The bay leaf is an aromatic leaf commonly used...,https://upload.wikimedia.org/wikipedia/commons...
4,bolivian coriander,https://en.wikipedia.org/wiki/Porophyllum_rude...,Porophyllum ruderale is an herbaceous annual p...,https://upload.wikimedia.org/wikipedia/commons...
5,borage,https://en.wikipedia.org/wiki/Borage,"Borage ( (listen); Borago officinalis), also k...",https://upload.wikimedia.org/wikipedia/commons...
6,chervil,https://en.wikipedia.org/wiki/Chervil,"Chervil (; Anthriscus cerefolium), sometimes c...",https://upload.wikimedia.org/wikipedia/commons...
7,chives,https://en.wikipedia.org/wiki/Chives,"Chives, scientific name Allium schoenoprasum, ...",https://upload.wikimedia.org/wikipedia/commons...
8,cicely,https://en.wikipedia.org/wiki/Cicely_Tyson,"Cicely L. Tyson (born December 19, 1924) is an...",https://upload.wikimedia.org/wikipedia/commons...
9,cilantro,https://en.wikipedia.org/wiki/Coriander,Coriander (; Coriandrum sativum) is an annual ...,https://upload.wikimedia.org/wikipedia/commons...
10,coriander,https://en.wikipedia.org/wiki/Coriander,Coriander (; Coriandrum sativum) is an annual ...,https://upload.wikimedia.org/wikipedia/commons...


In [29]:
df_check_herbs.drop([3,15,34],axis=0,inplace=True)
df_check_herbs.shape

(41, 4)

In [0]:
unresolved_herbs_list = ['culantro', 'kinh gioi', 'mint', 'sage', 'sansho', 'savory', 'woodruff','boldo', 'mitsuba','epazote']
herbs_disambiguated_list = ['Eryngium_foetidum','Elsholtzia_ciliata', 'Mentha_arvensis', 'Salvia_officinalis', 'Zanthoxylum_piperitum','Summer_savory','Asperula','Boldo','Cryptotaenia','Dysphania_ambrosioides']

In [0]:
undisambiguated_herbs_list = get_wikipedia_page_summaries(herbs_disambiguated_list, 'herbs_list_2.jsonl',lang='en')

In [38]:
df_check_herbs_2 = fix_dataframe('herbs_list_2.jsonl')
df_check_herbs_2 = pd.concat([pd.DataFrame(unresolved_herbs_list),df_check_herbs_2],axis=1)
df_check_herbs_2.columns = ['name','disambiguated','page','summary','image_ref']
df_check_herbs_2

0 []


,name,disambiguated,page,summary,image_ref
0,culantro,Eryngium_foetidum,https://en.wikipedia.org/wiki/Eryngium_foetidum,Eryngium foetidum is a tropical perennial herb...,https://upload.wikimedia.org/wikipedia/commons...
1,kinh gioi,Elsholtzia_ciliata,https://en.wikipedia.org/wiki/Elsholtzia_ciliata,"Elsholtzia ciliata, commonly known as Vietname...",https://upload.wikimedia.org/wikipedia/commons...
2,mint,Mentha_arvensis,https://en.wikipedia.org/wiki/Mentha_arvensis,"Mentha arvensis, the corn mint, field mint, or...",https://upload.wikimedia.org/wikipedia/commons...
3,sage,Salvia_officinalis,https://en.wikipedia.org/wiki/Salvia_officinalis,"Salvia officinalis (sage, also called garden s...",https://upload.wikimedia.org/wikipedia/commons...
4,sansho,Zanthoxylum_piperitum,https://en.wikipedia.org/wiki/Zanthoxylum_pipe...,"Zanthoxylum piperitum, also known as Japanese ...",https://upload.wikimedia.org/wikipedia/commons...
5,savory,Summer_savory,https://en.wikipedia.org/wiki/Summer_savory,Summer savory (Satureja hortensis) is among th...,https://upload.wikimedia.org/wikipedia/commons...
6,woodruff,Asperula,https://en.wikipedia.org/wiki/Asperula,"Asperula, commonly known as woodruff, is a gen...",https://upload.wikimedia.org/wikipedia/commons...
7,boldo,Boldo,https://en.wikipedia.org/wiki/Boldo,"""Boldo"" is also a settlement in Arauco Provinc...",https://upload.wikimedia.org/wikipedia/commons...
8,mitsuba,Cryptotaenia,https://en.wikipedia.org/wiki/Cryptotaenia,"Cryptotaenia, also honewort, is a genus of her...",https://upload.wikimedia.org/wikipedia/commons...
9,epazote,Dysphania_ambrosioides,https://en.wikipedia.org/wiki/Dysphania_ambros...,"Dysphania ambrosioides, formerly Chenopodium a...",https://upload.wikimedia.org/wikipedia/commons...


In [0]:
herbs_list = ["Angelica",
"Basil",
"Holy Basil",
"Thai Basil",
"Bay leaf",
"Indian Bay leaf",
"Boldo",
"Borage",
"Chervil",
"Chives",
"Garlic Chives",
"Cicely",
"Coriander",
"Cilantro",
"Bolivian Coriander",
"Vietnamese Coriander",
"Culantro",
"Cress",
"Curry leaf",
"Dill",
"Epazote",
"Hemp",
"Hoja santa",
"Houttuynia cordata",
"Hyssop",
"Jimbu",
"Kinh gioi",
"Lavender",
"Lemon balm",
"Lemon grass",
"Lemon myrtle",
"Lemon verbena",
"Limnophila aromatica",
"Lovage",
"Marjoram",
"Mint",
"Mugwort",
"Mitsuba",
"Oregano",
"Parsley",
"Perilla",
"Rosemary",
"Rue",
"Sage",
"Savory",
"Sansho",
"Shiso",
"Sorrel",
"Tarragon",
"Thyme",
"Woodruff"]


In [10]:
print(len(herbs_list),herbs_list)

51 ['Angelica', 'Basil', 'Holy Basil', 'Thai Basil', 'Bay leaf', 'Indian Bay leaf', 'Boldo', 'Borage', 'Chervil', 'Chives', 'Garlic Chives', 'Cicely', 'Coriander', 'Cilantro', 'Bolivian Coriander', 'Vietnamese Coriander', 'Culantro', 'Cress', 'Curry leaf', 'Dill', 'Epazote', 'Hemp', 'Hoja santa', 'Houttuynia cordata', 'Hyssop', 'Jimbu', 'Kinh gioi', 'Lavender', 'Lemon balm', 'Lemon grass', 'Lemon myrtle', 'Lemon verbena', 'Limnophila aromatica', 'Lovage', 'Marjoram', 'Mint', 'Mugwort', 'Mitsuba', 'Oregano', 'Parsley', 'Perilla', 'Rosemary', 'Rue', 'Sage', 'Savory', 'Sansho', 'Shiso', 'Sorrel', 'Tarragon', 'Thyme', 'Woodruff']


In [39]:
spices_list = [
"Aonori",
"Ajwain",
"Allspice",
"Amchoor",
"Anise",
"Star Anise",
"Asafoetida",
"Camphor",
"Caraway",
"Cardamom",
"Black Cardamom",
"Cassia",
"Celery powder",
"Celery seed",
"Charoli",
"Chenpi",
"Cinnamon",
"Clove",
"Coriander seed",
"Cubeb",
"Cumin",
"Nigella sativa",
"Bunium persicum",
"Dill",
"Dill seed",
"Fennel",
"Fenugreek",
"Fingerroot",
"Greater Galangal",
"Lesser Galangal",
"Garlic",
"Ginger",
"Aromatic Ginger",
"Golpar",
"Grains of Paradise",
"Grains of Selim",
"Horseradish",
"Juniper berry",
"Kokum",
"Korarima",
"Dried Lime",
"Liquorice",
"Litsea cubeba",
"Mace",
"Mango-ginger",
"Mastic",
"Mahlab",
"Black Mustard",
"Brown Mustard",
"White Mustard",
"Nigella",
"Njangsa",
"Nutmeg",
"Alligator Pepper",
"Brazilian Pepper",
"Chili Pepper",
"Cayenne pepper",
"Paprika",
"Long Pepper",
"Peruvian Pepper",
"East Asian Pepper",
"Sichuan Pepper",
"Sansho",
"Tasmanian Pepper",
"Black Peppercorn",
"Green Peppercorn",
"White Peppercorn",
"Pomegranate seed",
"Poppy seed",
"Radhuni",
"Rose",
"Saffron",
"Salt",
"Sarsaparilla",
"Sassafras",
"Sesame",
"Shiso",
"Sumac",
"Tamarind",
"Tonka bean",
"Turmeric",
"Uzazi",
"Vanilla",
"Voatsiperifery",
"Wasabi",
"Yuzu",
"Zedoary",
"Zereshk",
"Zest"
]
print(len(spices_list),spices_list)


89 ['Aonori', 'Ajwain', 'Allspice', 'Amchoor', 'Anise', 'Star Anise', 'Asafoetida', 'Camphor', 'Caraway', 'Cardamom', 'Black Cardamom', 'Cassia', 'Celery powder', 'Celery seed', 'Charoli', 'Chenpi', 'Cinnamon', 'Clove', 'Coriander seed', 'Cubeb', 'Cumin', 'Nigella sativa', 'Bunium persicum', 'Dill', 'Dill seed', 'Fennel', 'Fenugreek', 'Fingerroot', 'Greater Galangal', 'Lesser Galangal', 'Garlic', 'Ginger', 'Aromatic Ginger', 'Golpar', 'Grains of Paradise', 'Grains of Selim', 'Horseradish', 'Juniper berry', 'Kokum', 'Korarima', 'Dried Lime', 'Liquorice', 'Litsea cubeba', 'Mace', 'Mango-ginger', 'Mastic', 'Mahlab', 'Black Mustard', 'Brown Mustard', 'White Mustard', 'Nigella', 'Njangsa', 'Nutmeg', 'Alligator Pepper', 'Brazilian Pepper', 'Chili Pepper', 'Cayenne pepper', 'Paprika', 'Long Pepper', 'Peruvian Pepper', 'East Asian Pepper', 'Sichuan Pepper', 'Sansho', 'Tasmanian Pepper', 'Black Peppercorn', 'Green Peppercorn', 'White Peppercorn', 'Pomegranate seed', 'Poppy seed', 'Radhuni', 'Ro

In [40]:
spices_list = remove_duplicate_list_items(spices_list)
print(len(spices_list),spices_list)

89 ['ajwain', 'alligator pepper', 'allspice', 'amchoor', 'anise', 'aonori', 'aromatic ginger', 'asafoetida', 'black cardamom', 'black mustard', 'black peppercorn', 'brazilian pepper', 'brown mustard', 'bunium persicum', 'camphor', 'caraway', 'cardamom', 'cassia', 'cayenne pepper', 'celery powder', 'celery seed', 'charoli', 'chenpi', 'chili pepper', 'cinnamon', 'clove', 'coriander seed', 'cubeb', 'cumin', 'dill', 'dill seed', 'dried lime', 'east asian pepper', 'fennel', 'fenugreek', 'fingerroot', 'garlic', 'ginger', 'golpar', 'grains of paradise', 'grains of selim', 'greater galangal', 'green peppercorn', 'horseradish', 'juniper berry', 'kokum', 'korarima', 'lesser galangal', 'liquorice', 'litsea cubeba', 'long pepper', 'mace', 'mahlab', 'mango ginger', 'mastic', 'nigella', 'nigella sativa', 'njangsa', 'nutmeg', 'paprika', 'peruvian pepper', 'pomegranate seed', 'poppy seed', 'radhuni', 'rose', 'saffron', 'salt', 'sansho', 'sarsaparilla', 'sassafras', 'sesame', 'shiso', 'sichuan pepper',

In [41]:
unresolved_spices_list = get_wikipedia_page_summaries(spices_list, 'spices_list.jsonl',lang='en')
unresolved_spices_list

/usr/local/lib/python3.6/dist-packages/wikipedia/wikipedia.py:389: UserWarning: No parser was explicitly specified, so I'm using the best available HTML parser for this system ("lxml"). This usually isn't a problem, but if you run this code on another system, or in a different virtual environment, it may use a different parser and behave differently.

The code that caused this warning is on line 389 of the file /usr/local/lib/python3.6/dist-packages/wikipedia/wikipedia.py. To get rid of this warning, pass the additional argument 'features="lxml"' to the BeautifulSoup constructor.

  lis = BeautifulSoup(html).find_all('li')


['aonori', 'cassia', 'charoli', 'cubeb', 'mace', 'radhuni', 'sansho', 'zest']

In [42]:
df_check_spices = fix_dataframe('spices_list.jsonl')
df_check_spices.to_csv('df_check_spices.csv')

8 ['aonori', 'cassia', 'charoli', 'cubeb', 'mace', 'radhuni', 'sansho', 'zest']


In [44]:
df_check_spices.head(88)

,name,page,summary,image_ref
0,ajwain,https://en.wikipedia.org/wiki/Ajwain,"Ajwain, ajowan (), or Trachyspermum ammi—also ...",https://upload.wikimedia.org/wikipedia/commons...
1,alligator pepper,https://en.wikipedia.org/wiki/Alligator_pepper,Alligator pepper (also known as mbongo spice o...,https://upload.wikimedia.org/wikipedia/commons...
2,allspice,https://en.wikipedia.org/wiki/Allspice,"Allspice, also known as Jamaica pepper, myrtle...",https://upload.wikimedia.org/wikipedia/commons...
3,amchoor,https://en.wikipedia.org/wiki/Amchoor,"Amchoor or aamchur or amchur, also referred to...",https://upload.wikimedia.org/wikipedia/commons...
4,anise,https://en.wikipedia.org/wiki/Anise,"Anise (; Pimpinella anisum), also called anise...",https://upload.wikimedia.org/wikipedia/commons...
...,...,...,...,...
83,white mustard,https://en.wikipedia.org/wiki/White_mustard,White mustard (Sinapis alba) is an annual plan...,https://upload.wikimedia.org/wikipedia/commons...
84,white peppercorn,https://en.wikipedia.org/wiki/Black_pepper,Black pepper (Piper nigrum) is a flowering vin...,https://upload.wikimedia.org/wikipedia/commons...
85,yuzu,https://en.wikipedia.org/wiki/Yuzu,"Yuzu (Citrus junos, from Japanese ユズ or 柚子) is...",https://upload.wikimedia.org/wikipedia/commons...
86,zedoary,https://en.wikipedia.org/wiki/Curcuma_zedoaria,"Curcuma zedoaria (zedoary, white turmeric, or ...",https://upload.wikimedia.org/wikipedia/commons...


In [0]:
unresolved_spices_list = ['aonori', 'cassia', 'charoli', 'cubeb', \
                          'mace', 'radhuni', 'sansho', 'zest'\
                          ]
spices_disambiguated_list = ['Green_laver','Cinnamomum_cassia', 'Buchanania_lanzan', 'Piper_cubeba',\
                            'Nutmeg#Mace','Trachyspermum_roxburghianum','Zanthoxylum_piperitum','Zest_(ingredient)',\
                            'Cryptotaenia','Dysphania_ambrosioides']

In [0]:
undisambiguated_spices_list = get_wikipedia_page_summaries(spices_disambiguated_list, 'spices_list_2.jsonl',lang='en')

In [0]:
df_check_spices_2 = fix_dataframe('spices_list_2.jsonl')
df_check_spices_2 = pd.concat([pd.DataFrame(unresolved_spices_list),df_check_spices_2],axis=1)
df_check_spices_2.columns = ['name','disambiguated','page','summary','image_ref']
df_check_spices_2

In [48]:
condiments_list = [
"Adjika",
"Advieh",
"Baharat",
"Beau Monde seasoning",
"Berbere",
"Bouquet garni",
"Buknu",
"Chaat masala",
"Chaunk",
"Chili powder",
"Crab boil",
"Curry powder",
"Doubanjiang",
"Douchi",
"Duqqa",
"Fines herbes",
"Five-spice powder",
"Garam masala",
"Garlic powder",
"Garlic salt",
"Gochujang",
"Harissa",
"Hawaij",
"Herbes de Provence",
"Idli podi",
"Jamaican jerk spice",
"Khmeli suneli",
"Lemon pepper",
"Mitmita",
"Mixed spice",
"Montreal steak seasoning",
"Mulling spices",
"Old Bay Seasoning",
"Onion powder",
"Panch phoron",
"Persillade",
"Powder-douce",
"Pumpkin pie spice",
"Qâlat daqqa",
"Quatre épices",
"Ras el hanout",
"Recado rojo",
"Sharena sol",
"Shichimi",
"Tabil",
"Tandoori masala",
"Vadouvan",
"Yuzukosho",
"Za'atar"
]
print(len(condiments_list),condiments_list)

49 ['Adjika', 'Advieh', 'Baharat', 'Beau Monde seasoning', 'Berbere', 'Bouquet garni', 'Buknu', 'Chaat masala', 'Chaunk', 'Chili powder', 'Crab boil', 'Curry powder', 'Doubanjiang', 'Douchi', 'Duqqa', 'Fines herbes', 'Five-spice powder', 'Garam masala', 'Garlic powder', 'Garlic salt', 'Gochujang', 'Harissa', 'Hawaij', 'Herbes de Provence', 'Idli podi', 'Jamaican jerk spice', 'Khmeli suneli', 'Lemon pepper', 'Mitmita', 'Mixed spice', 'Montreal steak seasoning', 'Mulling spices', 'Old Bay Seasoning', 'Onion powder', 'Panch phoron', 'Persillade', 'Powder-douce', 'Pumpkin pie spice', 'Qâlat daqqa', 'Quatre épices', 'Ras el hanout', 'Recado rojo', 'Sharena sol', 'Shichimi', 'Tabil', 'Tandoori masala', 'Vadouvan', 'Yuzukosho', "Za'atar"]


In [49]:
condiments_list = remove_duplicate_list_items(condiments_list)
print(len(condiments_list),condiments_list)

49 ['adjika', 'advieh', 'baharat', 'beau monde seasoning', 'berbere', 'bouquet garni', 'buknu', 'chaat masala', 'chaunk', 'chili powder', 'crab boil', 'curry powder', 'doubanjiang', 'douchi', 'duqqa', 'fines herbes', 'five spice powder', 'garam masala', 'garlic powder', 'garlic salt', 'gochujang', 'harissa', 'hawaij', 'herbes de provence', 'idli podi', 'jamaican jerk spice', 'khmeli suneli', 'lemon pepper', 'mitmita', 'mixed spice', 'montreal steak seasoning', 'mulling spices', 'old bay seasoning', 'onion powder', 'panch phoron', 'persillade', 'powder douce', 'pumpkin pie spice', 'quatre épices', 'qâlat daqqa', 'ras el hanout', 'recado rojo', 'sharena sol', 'shichimi', 'tabil', 'tandoori masala', 'vadouvan', 'yuzukosho', "za'atar"]


In [50]:
unresolved_condiments_list = get_wikipedia_page_summaries(condiments_list, 'condiments_list.jsonl',lang='en')
unresolved_condiments_list

[]

In [51]:
df_check_condiments = fix_dataframe('condiments_list.jsonl')
df_check_condiments.to_csv('df_check_condiments.csv')

0 []


In [52]:
df_check_condiments.head(49)

,name,page,summary,image_ref
0,adjika,https://en.wikipedia.org/wiki/Adjara,Adjara (Georgian: აჭარა Ač’ara [at͡ʃʼara] (lis...,https://upload.wikimedia.org/wikipedia/commons...
1,advieh,https://en.wikipedia.org/wiki/Advieh,Advieh (Persian: ادویه‎) means spice in Persia...,https://upload.wikimedia.org/wikipedia/commons...
2,baharat,https://en.wikipedia.org/wiki/Baharat,Not to be confused with Bharat.\nBahārāt (Arab...,https://upload.wikimedia.org/wikipedia/commons...
3,beau monde seasoning,https://en.wikipedia.org/wiki/Beau_Monde_seaso...,Beau Monde seasoning is a seasoning mixture. B...,https://upload.wikimedia.org/wikipedia/commons...
4,berbere,https://en.wikipedia.org/wiki/Berbere,"Berbere (Amharic: በርበሬ bärbäre, Tigrinya: በርበረ...",https://upload.wikimedia.org/wikipedia/commons...
5,bouquet garni,https://en.wikipedia.org/wiki/Bouquet_garni,"The bouquet garni (French for ""garnished bouqu...",https://upload.wikimedia.org/wikipedia/commons...
6,buknu,https://en.wikipedia.org/wiki/Buknu,Buknu is a powdered mixture of several spices ...,https://upload.wikimedia.org/wikipedia/commons...
7,chaat masala,https://en.wikipedia.org/wiki/Chaat_masala,"Chaat masala, also spelled chat masala, is a s...",https://upload.wikimedia.org/wikipedia/commons...
8,chaunk,https://en.wikipedia.org/wiki/Chain,A chain is a serial assembly of connected piec...,https://upload.wikimedia.org/wikipedia/commons...
9,chili powder,https://en.wikipedia.org/wiki/Chili_powder,"Chili powder (also spelled chile, chilli, or, ...",https://upload.wikimedia.org/wikipedia/commons...


In [0]:
unresolved_condiments_list = ['adjika', 'chaunk']
condiments_disambiguated_list = ['ajika','Tempering_(spices)']

In [77]:
df_check_condiments.drop([0,8],axis=0,inplace=True)
df_check_condiments.shape

(47, 4)

In [0]:
undisambiguated_condiments_list = get_wikipedia_page_summaries(condiments_disambiguated_list, 'condiments_list_2.jsonl',lang='en')

In [55]:
df_check_condiments_2 = fix_dataframe('condiments_list_2.jsonl')
df_check_condiments_2 = pd.concat([pd.DataFrame(unresolved_condiments_list),df_check_condiments_2],axis=1)
df_check_condiments_2.columns = ['name','disambiguated','page','summary','image_ref']
df_check_condiments_2

0 []


,name,disambiguated,page,summary,image_ref
0,adjika,ajika,https://en.wikipedia.org/wiki/Ajika,"Ajika or adjika (Abkhazian: аџьыка, Georgian: ...",https://upload.wikimedia.org/wikipedia/commons...
1,chaunk,Tempering_(spices),https://en.wikipedia.org/wiki/Tempering_(spices),Tempering is a cooking technique used in the c...,https://upload.wikimedia.org/wikipedia/commons...


In [67]:
breads_list = remove_duplicate_list_items(breads_list)
print(len(breads_list),breads_list)

pastries_list = remove_duplicate_list_items(pastries_list)
print(len(pastries_list),pastries_list)

fruits_list = remove_duplicate_list_items(fruits_list)
print(len(fruits_list),fruits_list)

vegetables_list = remove_duplicate_list_items(vegetables_list)
print(len(vegetables_list),vegetables_list)

35 ['bagel', 'baguette', 'boule', 'brioche', 'casalinga', 'chapati', 'ciabatta', 'cornbread', 'crumpet', 'epi', 'ficelle', 'flatbread', 'focaccia', 'fruit bread', 'hamburger bun', 'hardtack', 'lavash', 'matzoh', 'muffin', 'naan', 'pain au levain', 'pain de mie', "pane d'olive", 'paratha', 'pita', 'potato bread', 'pretzel', 'pumpernickel', 'roti', 'rugbrød', 'rye', 'scone', 'soda bread', 'sourdough', 'tortilla']
20 ['baklava', 'bomboloni', 'canelé', 'cinnamon roll', 'croissant', 'cronut', 'doughnut', 'gulab jamun', 'kouign amann', 'mille feuille', 'morning bun', 'pain au chocolat', 'pastel de nata', 'profiterole', 'rugelach', 'shortcake', 'stollen', 'strudel', 'toaster strudel', 'éclair']
81 ['apple', 'apricot', 'avocado', 'banana', 'bell pepper', 'bilberry', 'blackberry', 'blackcurrant', 'blood orange', 'blueberry', 'boysenberry', 'breadfruit', 'canary melon', 'cantaloupe', 'cherimoya', 'cherry', 'chili pepper', 'clementine', 'cloudberry', 'coconut', 'cranberry', 'cucumber', 'currant',

In [68]:
unresolved_breads_list = get_wikipedia_page_summaries(breads_list, 'breads_list.jsonl',lang='en')
unresolved_breads_list

unresolved_pastries_list = get_wikipedia_page_summaries(pastries_list, 'pastries_list.jsonl',lang='en')
unresolved_pastries_list

unresolved_fruits_list = get_wikipedia_page_summaries(fruits_list, 'fruits_list.jsonl',lang='en')
unresolved_fruits_list

unresolved_vegetables_list = get_wikipedia_page_summaries(vegetables_list, 'vegetables_list.jsonl',lang='en')
unresolved_vegetables_list

/usr/local/lib/python3.6/dist-packages/wikipedia/wikipedia.py:389: UserWarning: No parser was explicitly specified, so I'm using the best available HTML parser for this system ("lxml"). This usually isn't a problem, but if you run this code on another system, or in a different virtual environment, it may use a different parser and behave differently.

The code that caused this warning is on line 389 of the file /usr/local/lib/python3.6/dist-packages/wikipedia/wikipedia.py. To get rid of this warning, pass the additional argument 'features="lxml"' to the BeautifulSoup constructor.

  lis = BeautifulSoup(html).find_all('li')


['boule', 'casalinga', 'epi', "pane d'olive"]

[]

['currant',
 'date',
 'fig',
 'honeydew',
 'lime',
 'mandarine',
 'nut',
 'orange',
 'plum',
 'satsuma']

['beet',
 'black bean',
 'calabrese',
 'frisee',
 'green pepper',
 'herbs and spice',
 'mangetout',
 'sage',
 'squash ',
 'yam']

In [70]:
df_check_breads = fix_dataframe('breads_list.jsonl')
df_check_pastries = fix_dataframe('pastries_list.jsonl')
df_check_fruits = fix_dataframe('fruits_list.jsonl')
df_check_vegetables = fix_dataframe('vegetables_list.jsonl')



4 ['boule', 'casalinga', 'epi', "pane d'olive"]
0 []
10 ['currant', 'date', 'fig', 'honeydew', 'lime', 'mandarine', 'nut', 'orange', 'plum', 'satsuma']
10 ['beet', 'black bean', 'calabrese', 'frisee', 'green pepper', 'herbs and spice', 'mangetout', 'sage', 'squash ', 'yam']


In [71]:
df_check_breads.head(35)
df_check_pastries.head(20)
df_check_fruits.head(81)
df_check_vegetables.head(120)

,name,page,summary,image_ref
0,bagel,https://en.wikipedia.org/wiki/Bagel,A bagel (Yiddish: בײגל‎ beygl; Polish: bajgiel...,https://upload.wikimedia.org/wikipedia/commons...
1,baguette,https://en.wikipedia.org/wiki/Baguette,"A baguette (; French: [baɡɛt]) is a long, thin...",https://upload.wikimedia.org/wikipedia/commons...
3,brioche,https://en.wikipedia.org/wiki/Brioche,"Brioche (, also UK: , US: , French: [bʁijɔʃ]) ...",https://upload.wikimedia.org/wikipedia/commons...
5,chapati,https://en.wikipedia.org/wiki/Chapati,"Chapati (alternatively spelled chapatti, chapp...",https://upload.wikimedia.org/wikipedia/commons...
6,ciabatta,https://en.wikipedia.org/wiki/Ciabatta,"Ciabatta (, Italian: [tʃaˈbatta]; literally ""s...",https://upload.wikimedia.org/wikipedia/commons...
7,cornbread,https://en.wikipedia.org/wiki/Cornbread,Cornbread is any quick bread containing cornme...,https://upload.wikimedia.org/wikipedia/commons...
8,crumpet,https://en.wikipedia.org/wiki/Crumpet,A crumpet (listen) is a small griddle cake ma...,https://upload.wikimedia.org/wikipedia/commons...
10,ficelle,https://en.wikipedia.org/wiki/Ficelle,"A ficelle is a type of French bread loaf, made...",https://upload.wikimedia.org/wikipedia/commons...
11,flatbread,https://en.wikipedia.org/wiki/Flatbread,"A flatbread is a bread made with flour, water,...",https://upload.wikimedia.org/wikipedia/commons...
12,focaccia,https://en.wikipedia.org/wiki/Focaccia,Focaccia (Italian pronunciation: [foˈkattʃa]; ...,https://upload.wikimedia.org/wikipedia/commons...


,name,page,summary,image_ref
0,baklava,https://en.wikipedia.org/wiki/Baklava,"Baklava (Arabic: بقلاوة‎; , , or ; Turkish pro...",https://upload.wikimedia.org/wikipedia/commons...
1,bomboloni,https://en.wikipedia.org/wiki/Bombolone,"A bombolone (Italian: [bomboˈloːne], pl. bombo...",https://upload.wikimedia.org/wikipedia/commons...
2,canelé,https://en.wikipedia.org/wiki/Canel%C3%A9,A canelé (French: [kan.le]) is a small French ...,https://upload.wikimedia.org/wikipedia/commons...
3,cinnamon roll,https://en.wikipedia.org/wiki/Cinnamon_roll,"A cinnamon roll (also cinnamon bun, cinnamon s...",https://upload.wikimedia.org/wikipedia/commons...
4,croissant,https://en.wikipedia.org/wiki/Croissant,"A croissant (UK: ; US: , ; French pronunciatio...",https://upload.wikimedia.org/wikipedia/commons...
5,cronut,https://en.wikipedia.org/wiki/Cronut,A Cronut is a croissant-doughnut pastry invent...,https://upload.wikimedia.org/wikipedia/commons...
6,doughnut,https://en.wikipedia.org/wiki/Doughnut,A doughnut or donut (the latter spelling ofte...,https://upload.wikimedia.org/wikipedia/commons...
7,gulab jamun,https://en.wikipedia.org/wiki/Gulab_jamun,Gulab jamun (also spelled gulaab jamun) is a m...,https://upload.wikimedia.org/wikipedia/commons...
8,kouign amann,https://en.wikipedia.org/wiki/Kouign-amann,Kouign-amann (pronounced [ˌkwiɲ aˈmãn]; pl. ko...,https://upload.wikimedia.org/wikipedia/commons...
9,mille feuille,https://en.wikipedia.org/wiki/Mille-feuille,The mille-feuille (French pronunciation: ​[mil...,https://upload.wikimedia.org/wikipedia/commons...


,name,page,summary,image_ref
0,apple,https://en.wikipedia.org/wiki/Apple,"An apple is a sweet, edible fruit produced by ...",https://upload.wikimedia.org/wikipedia/commons...
1,apricot,https://en.wikipedia.org/wiki/Apricot,"An apricot (US: (listen), UK: (listen)) is a...",https://upload.wikimedia.org/wikipedia/commons...
2,avocado,https://en.wikipedia.org/wiki/Avocado,"The avocado (Persea americana), a tree with pr...",https://upload.wikimedia.org/wikipedia/commons...
3,banana,https://en.wikipedia.org/wiki/Banana,A banana is an edible fruit – botanically a be...,https://upload.wikimedia.org/wikipedia/commons...
4,bell pepper,https://en.wikipedia.org/wiki/Bell_pepper,"The bell pepper (also known as sweet pepper, p...",https://upload.wikimedia.org/wikipedia/commons...
...,...,...,...,...
76,tamarillo,https://en.wikipedia.org/wiki/Tamarillo,The tamarillo is a small tree or shrub in the ...,https://upload.wikimedia.org/wikipedia/commons...
77,tangerine,https://en.wikipedia.org/wiki/Tangerine,"The tangerine (Citrus reticula L. var., someti...",https://upload.wikimedia.org/wikipedia/commons...
78,tomato,https://en.wikipedia.org/wiki/Tomato,"The tomato is the edible, often red, berry of ...",https://upload.wikimedia.org/wikipedia/commons...
79,ugli fruit,https://en.wikipedia.org/wiki/Jamaican_tangelo,"The Jamaican tangelo, also known by proprietar...",https://upload.wikimedia.org/wikipedia/commons...


,name,page,summary,image_ref
0,acorn squash,https://en.wikipedia.org/wiki/Acorn_squash,"Acorn squash (Cucurbita pepo var. turbinata), ...",https://upload.wikimedia.org/wikipedia/commons...
1,alfalfa sprout,https://en.wikipedia.org/wiki/Alfalfa,"Alfalfa (), also called lucerne and called Med...",https://upload.wikimedia.org/wikipedia/commons...
2,amaranth,https://en.wikipedia.org/wiki/Amaranth,"Amaranthus, collectively known as amaranth, is...",https://upload.wikimedia.org/wikipedia/commons...
3,anise,https://en.wikipedia.org/wiki/Anise,"Anise (; Pimpinella anisum), also called anise...",https://upload.wikimedia.org/wikipedia/commons...
4,artichoke,https://en.wikipedia.org/wiki/Artichoke,The globe artichoke (Cynara cardunculus var. s...,https://upload.wikimedia.org/wikipedia/commons...
...,...,...,...,...
114,wasabi,https://en.wikipedia.org/wiki/Wasabi,"Wasabi (Japanese: ワサビ, わさび or 山葵, pronounced [...",https://upload.wikimedia.org/wikipedia/commons...
115,water chestnut,https://en.wikipedia.org/wiki/Water_chestnut,Water chestnut may refer to either of two plan...,https://upload.wikimedia.org/wikipedia/commons...
116,watercress,https://en.wikipedia.org/wiki/Watercress,Watercress or yellowcress is an aquatic plant ...,https://upload.wikimedia.org/wikipedia/commons...
117,white radish,https://en.wikipedia.org/wiki/Daikon,"Daikon (大根, literally 'big root'), Raphanus sa...",https://upload.wikimedia.org/wikipedia/commons...


In [0]:
df_check_breads.to_csv('df_check_breads.csv')
df_check_pastries.to_csv('df_check_pastries.csv')
df_check_fruits.to_csv('df_check_fruits.csv')
df_check_vegetables.to_csv('df_check_vegetables.csv')

In [0]:
unresolved_breads_list = ['boule']
breads_disambiguated_list = ['Boule_(bread)']


unresolved_fruits_list = ['currant', 'date', 'fig', 'honeydew', 'lime', \
                          'mandarine', 'nut', 'orange', 'plum', 'satsuma']
fruits_disambiguated_list = ['Zante_currant','Date_palm#Fruits', 'Common_fig', 'Honeydew_(melon)', 'Lime_(fruit)',\
                             'Mandarin_orange', 'Nut_(fruit)', 'Orange_(fruit)', 'Plum', 'Citrus_unshiu']

unresolved_vegetables_list = ['beet', 'black bean', 'calabrese', 'frisee', 'green pepper', \
                              'mangetout', 'sage', 'squash ', 'yam']
vegetables_disambiguated_list = ['Beetroot','Vigna_mungo', 'Broccoli', 'Endive', 'Chili_pepper',\
                                 'Snap_pea', 'Salvia_officinalis', 'Cucurbita', 'Yam_(vegetable)']

In [0]:
undisambiguated_breads_list = get_wikipedia_page_summaries(breads_disambiguated_list, 'breads_list_2.jsonl',lang='en')
undisambiguated_fruits_list = get_wikipedia_page_summaries(fruits_disambiguated_list, 'fruits_list_2.jsonl',lang='en')
undisambiguated_vegetables_list = get_wikipedia_page_summaries(vegetables_disambiguated_list, 'vegetables_list_2.jsonl',lang='en')

In [76]:
df_check_breads_2 = fix_dataframe('breads_list_2.jsonl')
df_check_breads_2 = pd.concat([pd.DataFrame(unresolved_breads_list),df_check_breads_2],axis=1)
df_check_breads_2.columns = ['name','disambiguated','page','summary','image_ref']
df_check_breads_2

df_check_fruits_2 = fix_dataframe('fruits_list_2.jsonl')
df_check_fruits_2 = pd.concat([pd.DataFrame(unresolved_fruits_list),df_check_fruits_2],axis=1)
df_check_fruits_2.columns = ['name','disambiguated','page','summary','image_ref']
df_check_fruits_2

df_check_vegetables_2 = fix_dataframe('vegetables_list_2.jsonl')
df_check_vegetables_2 = pd.concat([pd.DataFrame(unresolved_vegetables_list),df_check_vegetables_2],axis=1)
df_check_vegetables_2.columns = ['name','disambiguated','page','summary','image_ref']
df_check_vegetables_2

0 []


,name,disambiguated,page,summary,image_ref
0,boule,Boule_(bread),https://en.wikipedia.org/wiki/Boule_(bread),"Boule, from the French for ""ball"", is a tradit...",https://upload.wikimedia.org/wikipedia/commons...


0 []


,name,disambiguated,page,summary,image_ref
0,currant,Zante_currant,https://en.wikipedia.org/wiki/Zante_currant,"Zante currants, Corinth raisins, or Corinthian...",https://upload.wikimedia.org/wikipedia/commons...
1,date,Date_palm#Fruits,https://en.wikipedia.org/wiki/Date_palm,"Phoenix dactylifera, commonly known as date or...",https://upload.wikimedia.org/wikipedia/commons...
2,fig,Common_fig,https://en.wikipedia.org/wiki/Common_fig,Ficus carica is an Asian species of flowering ...,https://upload.wikimedia.org/wikipedia/commons...
3,honeydew,Honeydew_(melon),https://en.wikipedia.org/wiki/Honeydew_(melon),"A honeydew melon, also known as a honeymelon, ...",https://upload.wikimedia.org/wikipedia/commons...
4,lime,Lime_(fruit),https://en.wikipedia.org/wiki/Lime_(fruit),"A lime (from French lime, from Arabic līma, fr...",https://upload.wikimedia.org/wikipedia/commons...
5,mandarine,Mandarin_orange,https://en.wikipedia.org/wiki/Mandarin_orange,"The mandarin orange (Citrus reticulata), also ...",https://upload.wikimedia.org/wikipedia/commons...
6,nut,Nut_(fruit),https://en.wikipedia.org/wiki/Nut_(fruit),A nut is a fruit composed of an inedible hard ...,https://upload.wikimedia.org/wikipedia/commons...
7,orange,Orange_(fruit),https://en.wikipedia.org/wiki/Orange_(fruit),The orange is the fruit of the citrus species ...,https://upload.wikimedia.org/wikipedia/commons...
8,plum,Plum,https://en.wikipedia.org/wiki/Plum,A plum is a fruit of the subgenus Prunus of th...,https://upload.wikimedia.org/wikipedia/commons...
9,satsuma,Citrus_unshiu,https://en.wikipedia.org/wiki/Citrus_unshiu,Citrus unshiu is a seedless and easy-peeling c...,https://upload.wikimedia.org/wikipedia/commons...


1 ['yam_(vegetable)']


,name,disambiguated,page,summary,image_ref
0,beet,Beetroot,https://en.wikipedia.org/wiki/Beetroot,The beetroot is the taproot portion of a beet ...,https://upload.wikimedia.org/wikipedia/commons...
1,black bean,Vigna_mungo,https://en.wikipedia.org/wiki/Vigna_mungo,"Vigna mungo, black gram, urad bean, urid bean,...",https://upload.wikimedia.org/wikipedia/commons...
2,calabrese,Broccoli,https://en.wikipedia.org/wiki/Broccoli,Broccoli is an edible green plant in the cabba...,https://upload.wikimedia.org/wikipedia/commons...
3,frisee,Endive,https://en.wikipedia.org/wiki/Endive,Endive () is a leaf vegetable belonging to the...,https://upload.wikimedia.org/wikipedia/commons...
4,green pepper,Chili_pepper,https://en.wikipedia.org/wiki/Chili_pepper,"The chili pepper (also chile, chile pepper, ch...",https://upload.wikimedia.org/wikipedia/commons...
5,mangetout,Snap_pea,https://en.wikipedia.org/wiki/Snap_pea,"The snap pea (Pisum sativum var. macrocarpon),...",https://upload.wikimedia.org/wikipedia/commons...
6,sage,Salvia_officinalis,https://en.wikipedia.org/wiki/Salvia_officinalis,"Salvia officinalis (sage, also called garden s...",https://upload.wikimedia.org/wikipedia/commons...
7,squash,Cucurbita,https://en.wikipedia.org/wiki/Cucurbita,Cucurbita (Latin for gourd) is a genus of herb...,https://upload.wikimedia.org/wikipedia/commons...
8,yam,NaN,NaN,NaN,NaN


In [0]:
breads_list = [
"bagel",
"baguette",
"boule",
"brioche",
"casalinga",
"chapati",
"ciabatta",
"cornbread",
"crumpet",
"epi",
"ficelle",
"flatbread",
"focaccia",
"fruit bread",
"hamburger bun",
"hardtack",
"lavash",
"matzoh",
"muffin",
"naan",
"pain au levain",
"pain de mie",
"pane d'olive",
"paratha",
"pita",
"potato bread",
"pretzel",
"pumpernickel",
"roti",
"rugbrød",
"rye",
"scone",
"soda bread",
"sourdough",
"tortilla"
]
pastries_list = [
    "baklava",
    "bomboloni",
    "canelé",
    "cinnamon roll",
    "croissant",
    "cronut",
    "doughnut",
    "éclair",
    "gulab jamun",
    "kouign-amann",
    "mille-feuille",
    "morning bun",
    "pain au chocolat",
    "pastel de nata",
    "profiterole",
    "rugelach",
    "shortcake",
    "stollen",
    "strudel",
    "toaster strudel"
    ]

In [0]:
fruits_list = [
"apple",
"apricot",
"avocado",
"banana",
"bell pepper",
"bilberry",
"blackberry",
"blackcurrant",
"blood orange",
"blueberry",
"boysenberry",
"breadfruit",
"canary melon",
"cantaloupe",
"cherimoya",
"cherry",
"chili pepper",
"clementine",
"cloudberry",
"coconut",
"cranberry",
"cucumber",
"currant",
"damson",
"date",
"dragonfruit",
"durian",
"eggplant",
"elderberry",
"feijoa",
"fig",
"goji berry",
"gooseberry",
"grape",
"grapefruit",
"guava",
"honeydew",
"huckleberry",
"jackfruit",
"jambul",
"jujube",
"kiwi fruit",
"kumquat",
"lemon",
"lime",
"loquat",
"lychee",
"mandarine",
"mango",
"mulberry",
"nectarine",
"nut",
"olive",
"orange",
"pamelo",
"papaya",
"passionfruit",
"peach",
"pear",
"persimmon",
"physalis",
"pineapple",
"plum",
"pomegranate",
"pomelo",
"purple mangosteen",
"quince",
"raisin",
"rambutan",
"raspberry",
"redcurrant",
"rock melon",
"salal berry",
"satsuma",
"star fruit",
"strawberry",
"tamarillo",
"tangerine",
"tomato",
"ugli fruit",
"watermelon"
]

In [0]:
vegetables_list = [
"acorn squash",
"alfalfa sprout",
"amaranth",
"anise",
"artichoke",
"arugula",
"asparagus",
"aubergine",
"azuki bean",
"banana squash",
"basil",
"bean sprout",
"beet",
"black bean",
"black-eyed pea",
"bok choy",
"borlotti bean",
"broad beans",
"broccoflower",
"broccoli",
"brussels sprout",
"butternut squash",
"cabbage",
"calabrese",
"caraway",
"carrot",
"cauliflower",
"cayenne pepper",
"celeriac",
"celery",
"chamomile",
"chard",
"chayote",
"chickpea",
"chives",
"cilantro",
"collard green",
"corn",
"corn salad",
"courgette",
"cucumber",
"daikon",
"delicata",
"dill",
"eggplant",
"endive",
"fennel",
"fiddlehead",
"frisee",
"garlic",
"gem squash",
"ginger",
"green bean",
"green pepper",
"habanero",
"herbs and spice",
"horseradish",
"hubbard squash",
"jalapeno",
"jerusalem artichoke",
"jicama",
"kale",
"kidney bean",
"kohlrabi",
"lavender",
"leek ",
"legume",
"lemon grass",
"lentils",
"lettuce",
"lima bean",
"mamey",
"mangetout",
"marjoram",
"mung bean",
"mushroom",
"mustard green",
"navy bean",
"new zealand spinach",
"nopale",
"okra",
"onion",
"oregano",
"paprika",
"parsley",
"parsnip",
"patty pan",
"pea",
"pinto bean",
"potato",
"pumpkin",
"radicchio",
"radish",
"rhubarb",
"rosemary",
"runner bean",
"rutabaga",
"sage",
"scallion",
"shallot",
"skirret",
"snap pea",
"soy bean",
"spaghetti squash",
"spinach",
"squash ",
"sweet potato",
"tabasco pepper",
"taro",
"tat soi",
"thyme",
"topinambur",
"tubers",
"turnip",
"wasabi",
"water chestnut",
"watercress",
"white radish",
"yam",
"zucchini"
    ]

In [0]:
df_herbs = pd.concat([df_check_herbs, df_check_herbs_2],axis=0,ignore_index=True, sort=False)
df_condiments = pd.concat([df_check_condiments, df_check_condiments_2],axis=0,ignore_index=True, sort=False)
df_breads = pd.concat([df_check_breads, df_check_breads_2],axis=0,ignore_index=True, sort=False)
df_fruits = pd.concat([df_check_fruits, df_check_fruits_2],axis=0,ignore_index=True, sort=False)
df_vegetables = pd.concat([df_check_vegetables, df_check_vegetables_2],axis=0,ignore_index=True, sort=False)

In [80]:
list(df_herbs['name'])

['angelica',
 'basil',
 'bay leaf',
 'bolivian coriander',
 'borage',
 'chervil',
 'chives',
 'cicely',
 'cilantro',
 'coriander',
 'cress',
 'curry leaf',
 'dill',
 'garlic chives',
 'hemp',
 'hoja santa',
 'holy basil',
 'houttuynia cordata',
 'hyssop',
 'indian bay leaf',
 'jimbu',
 'lavender',
 'lemon balm',
 'lemon grass',
 'lemon myrtle',
 'lemon verbena',
 'limnophila aromatica',
 'lovage',
 'marjoram',
 'mugwort',
 'oregano',
 'parsley',
 'perilla',
 'rosemary',
 'rue',
 'shiso',
 'sorrel',
 'tarragon',
 'thai basil',
 'thyme',
 'vietnamese coriander',
 'culantro',
 'kinh gioi',
 'mint',
 'sage',
 'sansho',
 'savory',
 'woodruff',
 'boldo',
 'mitsuba',
 'epazote']

In [0]:
herb_list = list(df_herbs['name'])
condiments_list = list(df_condiments['name'])
spices_list = list(df_check_spices['name'])
pastries_list = list(df_check_pastries['name'])
breads_list = list(df_check_breads['name'])
fruits_list = list(df_check_fruits['name'])
vegetables_list = list(df_check_vegetables['name'])



In [0]:
def create_dict_for_jsonl(input_list,label):
  output_list = []

  for item in input_list:
    pattern_list = []
    pattern_dict = {}
    id_str = ''
    for word in item.split():
      token_dict = {}
      token_dict['LOWER'] = word
      pattern_list.append(token_dict)
      id_str += '_' + word 

    pattern_dict['label'] = label
    pattern_dict['pattern'] = pattern_list
    if id_str[0] == '_':
      id_str = id_str[1:]
    pattern_dict['id'] = id_str

    output_list.append(pattern_dict)

  return output_list    

In [0]:
herb_jsonl_list = create_dict_for_jsonl(herb_list,'herb')
spices_jsonl_list = create_dict_for_jsonl(spices_list,'spice')
condiments_jsonl_list = create_dict_for_jsonl(condiments_list,'condiment')
breads_jsonl_list = create_dict_for_jsonl(breads_list,'bread')
pastries_jsonl_list = create_dict_for_jsonl(pastries_list,'pastry')
fruits_jsonl_list = create_dict_for_jsonl(fruits_list,'fruit')
vegetables_jsonl_list = create_dict_for_jsonl(vegetables_list,'vegetable')



In [0]:
np.savez("herb_keyword_patterns", np.array(herb_list))
np.savez("spice_keyword_patterns", np.array(spices_list))
np.savez("condiment_keyword_patterns", np.array(condiments_list))
np.savez("bread_keyword_patterns", np.array(breads_list))
np.savez("pastry_keyword_patterns", np.array(pastries_list))
np.savez("fruit_keyword_patterns", np.array(fruits_list))
np.savez("vegetable_keyword_patterns", np.array(vegetables_list))


In [0]:
def load_full_jsonl(input_list, output_filename):
  for item in input_list:
    dump_jsonl([item],output_filename, append=True)
  return



In [0]:
load_full_jsonl(herb_jsonl_list, 'pattern_herb.jsonl')
load_full_jsonl(spices_jsonl_list, 'pattern_spice.jsonl')
load_full_jsonl(condiments_jsonl_list, 'pattern_condiment.jsonl')
load_full_jsonl(breads_jsonl_list, 'pattern_bread.jsonl')
load_full_jsonl(pastries_jsonl_list, 'pattern_pastry.jsonl')
load_full_jsonl(fruits_jsonl_list, 'pattern_fruit.jsonl')
load_full_jsonl(vegetables_jsonl_list, 'pattern_vegetable.jsonl')



In [0]:
meal_types = ['Breakfast','Elevenses','Brunch','Lunch','Dinner','Supper', 'Tiffin']
meal_courses = ['Entrée',"Hors d'oeuvre",'Entremet', 'dessert']

local_types = ['thali', 'bento'] 
cuisine_types = ['chinese', 'indian', 'korean','malay']
taste_types = ['sweet','sour','salty','bitter','umami','astringent','spicy']

meal_preparation = ['fried','baked','sauteed','skewered','caramelized','steamed','fermented','roast','processed',]